In [ ]:
import urllib.request
import os
import tarfile

url="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath="IMDb/aclImdb_v1.tar.gz"
if not os.path.isfile(filepath):
    result=urllib.request.urlretrieve(url,filepath)
    print('downloaded:',result)

In [3]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

#將html的標籤刪除
import re
def rm_htmltags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text)

#讀取IMDb的檔案目錄
import os
def read_files(filetype):
    path = "IMDb/aclImdb/"
    file_list=[]

    positive_path=path + filetype+"/pos/"
    for f in os.listdir(positive_path):
        file_list+=[positive_path+f]
    
    negative_path=path + filetype+"/neg/"
    for f in os.listdir(negative_path):
        file_list+=[negative_path+f]
        
    print('read',filetype, 'files:',len(file_list))
       
    all_labels = ([1] * 12500 + [0] * 12500) 
    
    all_texts  = []
    for fi in file_list:
        with open(fi,encoding='utf8') as file_input:
            all_texts += [rm_htmltags(" ".join(file_input.readlines()))]
            
    return all_labels,all_texts
    
#查看IMDb的檔案目錄

In [4]:
y_train, train_text = read_files("train")
y_test, test_text = read_files("test")

read train files: 25000
read test files: 25000


In [5]:
#先讀取所有文章建立dic，限制dic數量為nb_words=2500
token = Tokenizer(num_words=2500)
token.fit_on_texts(train_text)
#查看文字index的屬性

In [6]:
from keras.preprocessing import sequence 

x_train_seq = token.texts_to_sequences(train_text)
x_test_seq  = token.texts_to_sequences(test_text)

print(train_text[0])
print(x_train_seq[0])

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!
[308, 6, 3, 1068, 208, 8, 2160, 29, 1, 168, 54, 13, 45, 81, 40, 391, 109, 137, 13, 57, 149, 7, 1, 481, 68, 5, 260, 11, 2000, 6, 72, 2422, 5, 631, 70, 6, 1, 5, 2001, 1, 1530, 33, 66, 63, 204, 13

In [7]:
# set model
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers.embeddings import Embedding

model = Sequential()
model.add(Embedding(output_dim=32,
                    input_dim=2000, 
                    input_length=100))
model.add(Dropout(0.2))

In [8]:
model.add(Flatten())
model.add(Dense(units=256,
                activation='relu' ))
model.add(Dropout(0.2))
model.add(Dense(units=1,
                activation='sigmoid' ))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 32)           64000     
_________________________________________________________________
dropout (Dropout)            (None, 100, 32)           0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               819456    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 883,713
Trainable params: 883,713
Non-trainable params: 0
__________________________________________________

In [ ]:
# 利用反向演算法訓練模型

In [ ]:
x_train = sequence.pad_sequences(x_train_seq, maxlen=100)
x_test = sequence.pad_sequences(x_test_seq, maxlen=100)

In [9]:
model.compile(loss='binary_crossentropy',metrics=['accuracy'])
#進行訓練
#batch_size：每一批次訓練100筆資料
#epochs：執行10個訓練週期
#verbose：顯示每次的訓練過程
#validation_split：測試資料的比例
train_history =model.fit(x_train, y_train,batch_size=100,
                         epochs=10,verbose=2,validation_split=0.25)
                      
#評估訓練模型的準確率
acu = model.evaluate(x_test, y_test, verbose=1)
acu[1]

NameError: name 'x_train' is not defined

In [ ]:
# 建立RNN模型
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

model = Sequential()
model.add(Embedding(output_dim=32,
                    input_dim=3800, 
                    input_length=380))
model.add(Dropout(0.25))
#建立18個RNN神經元
model.add(SimpleRNN(units=18))
model.add(Dense(units=256,activation='relu' ))
model.add(Dropout(0.25))
model.add(Dense(units=1,activation='sigmoid' ))
model.summary()

In [ ]:
# 預測模型的準確率
scores = model.evaluate(x_test, y_test, verbose=1)
scores[1]